https://gdoria.com/A-Neural-Network-Classifier-with-Keras-and-Doc2Vec.html  
https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

In [9]:
import pandas as pd
import gensim
from gensim.models import Doc2Vec
from gensim import utils
import gensim.parsing.preprocessing as gsp
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
import numpy as np
from tensorflow.keras import utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import load_model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import plot_confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, RepeatVector
from keras.optimizers import SGD
from keras.models import Sequential

from nltk.tokenize import word_tokenize
    
bbc_text_df = pd.read_csv('data/bbc-text.csv')
bbc_text_df.head()
bbc_text = bbc_text_df.text.to_list()

In [10]:
le = preprocessing.LabelEncoder()

In [11]:
bbc_text_df['label'] = le.fit_transform(bbc_text_df.category)

In [12]:
le.fit(bbc_text_df['label'])
encoded_y = utils.to_categorical((le.transform(bbc_text_df['label'])))
print(encoded_y[0])

[0. 0. 0. 0. 1.]


In [13]:
bbc_text_df.head()

,category,text,label
0,tech,tv future in the hands of viewers with home th...,4
1,business,worldcom boss left books alone former worldc...,0
2,sport,tigers wary of farrell gamble leicester say ...,3
3,sport,yeading face newcastle in fa cup premiership s...,3
4,entertainment,ocean s twelve raids box office ocean s twelve...,1


In [14]:

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean_text(s):
    s = s.lower()
    s = gensim.utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

In [17]:
tagged_data = [TaggedDocument(words=word_tokenize(clean_text(_d)), tags=[str(i)]) for i, _d in enumerate(bbc_text)]

In [18]:
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    if epoch % 25 == 0:
        print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 25
iteration 50
iteration 75
Model Saved


In [31]:
train_size = model.corpus_count

In [34]:
text_train_arrays = np.zeros((model.corpus_count, vec_size))

for i in range(train_size):
    text_train_arrays[i] = model.docvecs[i]
    

In [54]:
def create_keras_model():
    keras_model = keras.Sequential(
        [
            keras.Input(shape=vec_size),
            layers.Dense(256,activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(256,activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(80,activation='relu'),
            layers.Dense(5, activation="softmax"),
        ]
    )
    keras_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"])
    return keras_model


In [55]:
keras_model = create_keras_model()

In [56]:
estimator=keras_model.fit(text_train_arrays, encoded_y, validation_split=0.2, epochs=10, batch_size=64)

Epoch 1/10
28/28 [==============================] - 0s 6ms/step - loss: 0.5003 - categorical_accuracy: 0.3438 - val_loss: 0.3511 - val_categorical_accuracy: 0.7281
Epoch 2/10
28/28 [==============================] - 0s 4ms/step - loss: 0.2966 - categorical_accuracy: 0.6635 - val_loss: 0.1439 - val_categorical_accuracy: 0.8921
Epoch 3/10
28/28 [==============================] - 0s 4ms/step - loss: 0.1418 - categorical_accuracy: 0.8635 - val_loss: 0.0790 - val_categorical_accuracy: 0.9079
Epoch 4/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0905 - categorical_accuracy: 0.9101 - val_loss: 0.0638 - val_categorical_accuracy: 0.9461
Epoch 5/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0562 - categorical_accuracy: 0.9466 - val_loss: 0.0786 - val_categorical_accuracy: 0.9303
Epoch 6/10
28/28 [==============================] - 0s 4ms/step - loss: 0.0463 - categorical_accuracy: 0.9562 - val_loss: 0.0738 - val_categorical_accuracy: 0.9326
Epoch 7/10
28/28

In [40]:
keras_model.save('keras.h5')

In [64]:
text_train_arrays[0].shape

(300,)

In [85]:
# keras_model.summary()

In [84]:
keras_model.predict(text_train_arrays[4].reshape(1,-1)).argmax()

1

In [80]:
bbc_text_df.head()

,category,text,label
0,tech,tv future in the hands of viewers with home th...,4
1,business,worldcom boss left books alone former worldc...,0
2,sport,tigers wary of farrell gamble leicester say ...,3
3,sport,yeading face newcastle in fa cup premiership s...,3
4,entertainment,ocean s twelve raids box office ocean s twelve...,1
